# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking current working directory
print(f"Current working directory : {os.getcwd()}")

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

Current working directory : C:\Users\randas\Desktop\Data_modeling_cassandra
['C:\\Users\\randas\\Desktop\\Data_modeling_cassandra/event_data\\2018-11-01-events.csv', 'C:\\Users\\randas\\Desktop\\Data_modeling_cassandra/event_data\\2018-11-02-events.csv', 'C:\\Users\\randas\\Desktop\\Data_modeling_cassandra/event_data\\2018-11-03-events.csv', 'C:\\Users\\randas\\Desktop\\Data_modeling_cassandra/event_data\\2018-11-04-events.csv', 'C:\\Users\\randas\\Desktop\\Data_modeling_cassandra/event_data\\2018-11-05-events.csv', 'C:\\Users\\randas\\Desktop\\Data_modeling_cassandra/event_data\\2018-11-06-events.csv', 'C:\\Users\\randas\\Desktop\\Data_modeling_cassandra/event_data\\2018-11-07-events.csv', 'C:\\Users\\randas\\Desktop\\Data_modeling_cassandra/event_data\\2018-11-08-events.csv', 'C:\\Users\\randas\\Desktop\\Data_modeling_cassandra/event_data\\2018-11-09-events.csv', 'C:\\Users\\randas\\Desktop\\Data_modeling_cassandra/event_data\\2018-11-10-events.csv', 'C:\\Users\\randas\\Desktop\\Data

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

print(f"Total rows : {len(full_data_rows_list)}")
print(f"Sample data:\n {full_data_rows_list[:5]}")

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Total rows : 8056
Sample data:
 [['', 'Logged In', 'Walter', 'M', '0', 'Frye', '', 'free', 'San Francisco-Oakland-Hayward, CA', 'GET', 'Home', '1.54092E+12', '38', '', '200', '1.54111E+12', '39'], ['', 'Logged In', 'Kaylee', 'F', '0', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Home', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ["Des'ree", 'Logged In', 'Kaylee', 'F', '1', 'Summers', '246.30812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'You Gotta Be', '200', '1.54111E+12', '8'], ['', 'Logged In', 'Kaylee', 'F', '2', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Upgrade', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ['Mr Oizo', 'Logged In', 'Kaylee', 'F', '3', 'Summers', '144.03873', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'Flat 55', '200', '1.54111E+12', '8']]


In [5]:
# checking the number of rows in new event csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [86]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("Connection Established !!")
except Exception as e:
    print(f"Connection Failed !! Error : {e}")

Connection Established !!


#### Creating Keyspace

In [7]:
keyspace_query = """CREATE KEYSPACE IF NOT EXISTS sparkify
                    with REPLICATION = 
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                """

# Creating Keyspace
try:
    session.execute(keyspace_query)
except Exception as e:
    print(f"Failed to create keyspace!! Error : {e}")

#### Setting Keyspace

In [88]:
# Setting KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra we model the database tables on the queries we want to run.

## Below are the queries following which we will build out data model

### 1. Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# ========================================================================================================

In [9]:
# Creating table for query1 
create_query1 = """CREATE TABLE IF NOT EXISTS artists_song_title_length (artistName text,songTitle text,songLength float, sessionId int,itemInSession int, PRIMARY KEY(sessionId,itemInSession))"""

try : 
    session.execute(create_query1)
except Exception as e : 
    print(e)


In [10]:
# Using the event file
file = 'event_datafile_new.csv'

# Reading csv file and inserting rows into cassandra tables.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
       #print(line)
       session.execute("""Insert into sparkify.artists_song_title_length(artistName,songTitle,songLength,sessionId,itemInSession)VALUES(%s,%s,%s,%s,%s)""",(line[0],line[-2],float(line[-6]),int(line[-3]),int(line[3])))


#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# SELECT statement to verify the data was entered into the table
select_query1 = """Select * From sparkify.artists_song_title_length where sessionId = 338 and itemInSession = 4"""
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(sessionid=338, iteminsession=4, artistname='Faithless', songlength=495.30731201171875, songtitle='Music Matters (Mark Knight Dub)')


# Creating table for query2 
# ### 2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [41]:

create_query2 = """CREATE TABLE IF NOT EXISTS artists_song_table"""
full_query2 = create_query2 + """(sessionId int,userId int,artist text,song text,firstName text,lastName text, itemInSession int, PRIMARY KEY ((sessionId, userId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC) """
try: 
    session.execute(full_query2)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


artist
firstName of user
gender of user
item number in session
last name of user
length of the song
level (paid or free song)
location of the user
sessionId
song title
userId

In [43]:
file = 'event_datafile_new.csv'
query = "INSERT INTO artists_song_table (sessionId, userId, artist, song, firstName, lastName, itemInSession) "
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s) "

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(query, (int(line[8]), int(line[10]), line[0], line[9], line[1], line[4], int(line[3])  ))
        except Exception as e:
            print(f"Faild to insert  : {e}")


In [45]:
# SELECT statement to verify the data was entered into the table

select_query2 = "SELECT artist, song, firstName, lastName FROM  artists_song_table where sessionId = 182 and userId = 10"
try:
    rows = session.execute(select_query2)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## Query 3

3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own

In [90]:
# Creating table for query3

create_query3 = """CREATE TABLE IF NOT EXISTS user_song"""
full_query3= create_query3 + """(firstName text,secondName text, song text, userId int, PRIMARY KEY ((song), userId))"""                                
try: 
    session.execute(full_query3)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


In [110]:
file = 'event_datafile_new.csv'
query_3 = "INSERT INTO user_song (firstName,secondName,song,userId) "
query_3 = query_3 + " VALUES (%s, %s, %s, %s) "
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query_3, (  line[1],line[4],line[9],int(line[10]) )  )

In [83]:
# SELECT statement to verify the data was entered into the table
select_query_3 = """Select * From user_song where song = 'All Hands Against His Own' """
res = session.execute(select_query_3)
for row in res : 
    print(row)

Row(song='All Hands Against His Own', userid=29, firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', userid=80, firstname='Tegan', lastname='Levine')
Row(song='All Hands Against His Own', userid=95, firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [84]:
session.execute("DROP TABLE IF EXISTS sparkify.artists_song_title_length")
session.execute("DROP TABLE IF EXISTS sparkify.artists_song_table")
session.execute("DROP TABLE IF EXISTS sparkify.user_song")

### Close the session and cluster connection¶

In [85]:
session.shutdown()
cluster.shutdown()